In [ ]:
!pip install numpy==1.23.5
!pip install xgboost==1.7.2
!pip install tensorflow==2.11.0
!pip install pygam==.0.8.0
!pip install piml
!pip install sklearn

In [ ]:

import pandas as pd
from google.colab import files
import pandas as pd
import numpy as np
uploaded = files.upload()
import io
df = pd.read_csv(io.BytesIO(uploaded['CaliforniaHousing.csv']), index_col = 0) # first␣column as index
df.describe(include='all')


Saving CaliforniaHousing.csv to CaliforniaHousing (1).csv


,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,28.639486,5.429000,1.096675,1425.476744,3.070655,35.631861,-119.569704,2.068558
std,12.585558,2.474173,0.473911,1132.462122,10.386050,2.135952,2.003532,1.153956
min,1.000000,0.846154,0.333333,3.000000,0.692308,32.540000,-124.350000,0.149990
25%,18.000000,4.440716,1.006079,787.000000,2.429741,33.930000,-121.800000,1.196000
50%,29.000000,5.229129,1.048780,1166.000000,2.818116,34.260000,-118.490000,1.797000
75%,37.000000,6.052381,1.099526,1725.000000,3.282261,37.710000,-118.010000,2.647250
max,52.000000,141.909091,34.066667,35682.000000,1243.333333,41.950000,-114.310000,5.000010


In [ ]:
!pip install interpret
!pip install sklearn


In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from interpret.glassbox import ExplainableBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV




#target variable
y = df['MedHouseVal']

X = df.drop(['MedHouseVal'],axis=1)

#test train split 70 / 30
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=0)



In [ ]:

# Define the EBM model object
ebm = ExplainableBoostingRegressor()

###*** KEEP IN MIND THESE ARE THE HYPERPARMETERS YOU CHOSE
### Adding too many hyper parameters results in a large search space which increases complexity of the search making it really computationally heavy
### adding too many hyper parameters could lead to the risk of over fitting your model which would then inturn decrease perfomrance
### adding too few hyperparameters could limit the search and lead to a under fitting or sub optimal performance of the model


# Define the hyperparameters to search over
params = {
    
    "max_bins": [5,10,15],
    "learning_rate": [0.01, 0.1,1],
    "interactions": [0, 1, 2,3],
    "max_leaves": [1, 2, 3,4]
    
}

# Perform grid search using 5-fold cross-validation 
grid_search = GridSearchCV(estimator=ebm, param_grid=params, cv=5)

# Fit the grid search to the training data
grid_search.fit(X_train, y_train)

#this is the best hyperparameters given the features you want to use
# Print the best hyperparameters and model performance 
print("Best parameters: ", grid_search.best_params_)
print("Best training score: ", grid_search.best_score_)
print("Test score: ", grid_search.score(X_test, y_test))

Best parameters:  {'interactions': 3, 'learning_rate': 1, 'max_bins': 15, 'max_leaves': 3}
Best training score:  0.7524567176420611
Test score:  0.7467305554158514


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Define the EBM model object
ebm2 = ExplainableBoostingRegressor()


# Define the hyperparameters to search over
# as per last Mondays meeting discussed I will be limiting hyper parameters just to bins and iteractions
params = {
    
    "max_bins": [5,10,15], 
    "interactions": [0, 1, 2,3],

    
}

# Perform grid search using 5-fold cross-validation 
grid_search = GridSearchCV(estimator=ebm2, param_grid=params, cv=5)

# Fit the grid search to the training data
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and model performance 
print("Best parameters: ", grid_search.best_params_)
print("Best training score: ", grid_search.best_score_)
print("Test score: ", grid_search.score(X_test, y_test))

Best parameters:  {'interactions': 3, 'max_bins': 15}
Best training score:  0.74357881800047
Test score:  0.7370196581521915


In [ ]:
##LGBM
import lightgbm as lgb


# Define the LightGBM model
lgbm = lgb.LGBMRegressor()

# Define the hyperparameters to search over
params = {
    "num_leaves": [10, 20, 30],
    "max_depth": [5, 10, 15],
    "learning_rate": [0.01, 0.1, 1],
    "n_estimators": [100, 200, 300],
    "reg_alpha": [0.1, 0.5, 1],
    "reg_lambda": [0.1, 0.5, 1]
}

# Perform grid search using 5-fold cross-validation
grid_search = GridSearchCV(estimator=lgbm, param_grid=params, cv=5)

# Fit the grid search to the training data
grid_search.fit(X, y)

# Print the best hyperparameters and model performance
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)


Best parameters:  {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300, 'num_leaves': 10, 'reg_alpha': 0.1, 'reg_lambda': 1}
Best score:  0.594016502215154


In [ ]:
##THIS IS A GRIDSEARCH FOR LGBM USING JUST 2 HYPERPARAMETERS
# Define the LightGBM model
lgbm = lgb.LGBMRegressor()

# Define the hyperparameters to search over
params = {

    "max_depth": [5, 10, 15],
    "n_estimators": [100, 200, 300],

}

# Perform grid search using 5-fold cross-validation
grid_search = GridSearchCV(estimator=lgbm, param_grid=params, cv=5)

# Fit the grid search to the training data
grid_search.fit(X, y)

# Print the best hyperparameters and model performance
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)


Best parameters:  {'max_depth': 15, 'n_estimators': 300}
Best score:  0.5706638479286804


In [ ]:
#### USING MSE as a scoring metric

In [ ]:
##EBM
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer, mean_squared_error
import numpy as np

ebm3 = ExplainableBoostingRegressor()


# Define the hyperparameters to search over
# as per last Mondays meeting discussed I will be limiting hyper parameters just to bins and iteractions
params = {
    
    "max_bins": [5,10,15], 
    "interactions": [0, 1, 2,3],

    
}

# Perform grid search using 5-fold cross-validation 
grid_search = GridSearchCV(estimator=ebm3, param_grid=params, cv=5,scoring=make_scorer(mean_squared_error, greater_is_better=False)
)

# Fit the grid search to the training data
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and model performance 
print("Best parameters: ", grid_search.best_params_)
print("Best training score: ", grid_search.best_score_)
print("Test score: ", grid_search.score(X_test, y_test))

Best parameters:  {'interactions': 3, 'max_bins': 15}
Best training score:  -0.34136117942083605
Test score:  -0.3506150685063607


In [ ]:
###LGB

##THIS IS A GRIDSEARCH FOR LGBM USING JUST 2 HYPERPARAMETERS
# Define the LightGBM model
lgbm = lgb.LGBMRegressor()

# Define the hyperparameters to search over
params = {

    "max_depth": [5, 10, 15],
    "n_estimators": [100, 200, 300],

}

# Perform grid search using 5-fold cross-validation
grid_search = GridSearchCV(estimator=lgbm, param_grid=params, cv=5, scoring=make_scorer(mean_squared_error, greater_is_better=False))

# Fit the grid search to the training data
grid_search.fit(X, y)

# Print the best hyperparameters and model performance
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)


Best parameters:  {'max_depth': 15, 'n_estimators': 300}
Best score:  -0.5342962248033933
